In [1]:
import pandas as pd

# Text processing and NLP
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import numpy as np

from tpot import TPOTClassifier

C:\Users\dross\AppData\Local\Temp\ipykernel_10916\3378462760.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\dross\Desktop\Project\venv\Lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


prior to starting will need to pip install the following 

pip install pandas scikit-learn numpy tpot

LOCALLY

GitHub

In [2]:
# df = pd.read_csv('C:/Users/dross/Desktop/Project/cleaned_whole_data.csv')
#tfidf_matrix = load_npz('C:/Users/dross/Desktop/Project/tfidf_matrix.npz')
# list(df['link_flair_text'].unique())
# labels = df['link_flair_text']
df = pd.read_csv('cleaned_whole_data.csv')
df.shape

(7897, 17)

Subreddit post distribution 

In [4]:
df['subreddit'].value_counts()

subreddit
careeradvice           973
LegalAdviceOffTopic    908
ITCareerQuestions      898
teachers               875
FinancialCareers       824
careerguidance         689
resumes                640
jobs                   616
cscareerquestions      522
sales                  448
AskHR                  421
EngineeringCareers      83
Name: count, dtype: int64

In [5]:
label_encoder = LabelEncoder()

# Fit and transform the 'subreddit' column to obtain encoded target variable
df['subreddit_encoded'] = label_encoder.fit_transform(df['subreddit'])


Convert to weekday and weekend One-hot encode 'day_of_week' and 'month', obtaining a new DataFrame

In [6]:

day_mapping = {
    "Monday": 1,
    "Tuesday": 2,
    "Wednesday": 3,
    "Thursday": 4,
    "Friday": 5,
    "Saturday": 6,
    "Sunday": 7
}

df['day_of_week_encoded'] = df['day_of_week'].map(day_mapping)

month_mapping = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12
}

df['month_encoded'] = df['month'].map(month_mapping)


df['is_weekday'] = df['day_of_week'].apply(lambda x: 1 if x in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'] else 0)
df.head()

,id,text,title,author,num_comments,post_id,upvote_ratio,score,url,subreddit,...,link_flair_template_id,created_datetime,day_of_week,hour_of_day,month,year,subreddit_encoded,day_of_week_encoded,month_encoded,is_weekday
0,41034,"Hi all, for context, 29 year old guy from Amst...",Lazy job or Hard job?,Weak_Assumption_6889,8,1bfpxll,0.33,0,https://www.reddit.com/r/careeradvice/comments...,careeradvice,...,Unknown,2024-03-15 22:07:22,Friday,22,March,2024,5,5,3,1
1,43519,Looking for a new role and haven\u2019t had mu...,Roast my Resume Pls,Neither_Trash,1,1bh8md2,0.99,1,https://i.redd.it/n918fjprlyoc1.jpeg,resumes,...,c292b8e0-28b9-11ec-874c-325b17e851a3,2024-03-17 21:05:40,Sunday,21,March,2024,9,7,3,0
2,41067,"I am a 24M, from west bengal with a bachelor's...",Is Jadavpur University good for MCA?,grvx_rdt,0,1bfg926,0.66,1,https://www.reddit.com/r/careeradvice/comments...,careeradvice,...,Unknown,2024-03-15 15:12:00,Friday,15,March,2024,5,5,3,1
3,43536,It's created in MS word.. I have 3.5 yoe in Ja...,Review my resume please,GroundbreakingZone94,7,1bh3jpg,0.60,1,https://i.redd.it/tjnti8vplxoc1.jpeg,resumes,...,3d212a9a-f5cb-11ec-acef-1adb3b338801,2024-03-17 17:43:39,Sunday,17,March,2024,9,7,3,0
4,43602,I live in Los Angeles and am 27 years old.\n,Thoughts on My Experience?,AshamedJellyfish9197,1,1bgejk2,0.33,0,https://i.redd.it/ifqv15ry3roc1.jpeg,resumes,...,c292b8e0-28b9-11ec-874c-325b17e851a3,2024-03-16 19:53:24,Saturday,19,March,2024,9,6,3,0


#to provide more context concatenated text and title then preformed tfidf on the Concatenated title and text

In [7]:
#to provide more context concatenated text and title then prefrom tfidf
# Concatenate title and text
df['combined_text'] = df['title'] + ' ' + df['text']

# Apply TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_text'])

# If you're using sparse matrices and want to combine with encoded_features, you might need to convert them to a dense format
tfidf_dense = tfidf_matrix.toarray()

We combine the dense array with the array_features for one set of features 

In [8]:
#Inorder to cobine all features will have to convert them to an numpy array 
#this dense matrix gives a 7% higher accuracy score but is expensive computional wise 
#solely due to cost ran sparse matrix  but feel free to run either

array_features = df[['num_comments','upvote_ratio','score','hour_of_day','year','day_of_week_encoded','month_encoded','is_weekday']].values

combined_features = np.hstack((tfidf_dense, array_features))


In [ ]:
#target varibale
y = df['subreddit_encoded'].values

In [14]:
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

In [ ]:
#Set up tpot classifier with generation 5 for iterations 

In [15]:
# tpot = TPOTClassifier(generations=10,population_size=100,verbosity=2,random_state=42, config_dict='TPOT sparse')
tpot = TPOTClassifier(generations=5, population_size=100, verbosity=2, random_state=42)

This is an expensive run would not run locally you can still use the suggested Linaer SVC on the next comment

In [16]:
tpot.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/600 [00:00<?, ?pipeline/s]

: 

In [14]:
tpot.export('best_tpot_pipelineV2.py')